# Cheat Sheet: Clasificacion de Teratogenicidad con Regresion Logistica

Este cuaderno expande el ejercicio original incorporando explicaciones paso a paso, mejores practicas y codigo comentado para replicar un flujo completo de machine learning supervisado en un problema de clasificacion binaria.

## Indice de Acciones
- [0. Contexto del Problema](#0-contexto-del-problema)
- [1. Preparacion del Entorno](#1-preparacion-del-entorno)
  - [1.1. Importacion de Librerias](#11-importacion-de-librerias)
  - [1.2. Configuracion General](#12-configuracion-general)
- [2. Comprension y Limpieza de Datos](#2-comprension-y-limpieza-de-datos)
  - [2.1. Carga y Exploracion Inicial](#21-carga-y-exploracion-inicial)
  - [2.2. Preparacion de Matrices X e y](#22-preparacion-de-matrices-x-e-y)
- [3. Preprocesamiento](#3-preprocesamiento)
  - [3.1. Variance Threshold](#31-variance-threshold)
  - [3.2. Estandarizacion](#32-estandarizacion)
- [4. Split de Entrenamiento y Test](#4-split-de-entrenamiento-y-test)
- [5. Modelo 1: Logistic Regression con Lasso](#5-modelo-1-logistic-regression-con-lasso)
- [6. Modelo 2: Logistic Regression con Elastic Net](#6-modelo-2-logistic-regression-con-elastic-net)
- [7. Seleccion de Features con SelectKBest](#7-seleccion-de-features-con-selectkbest)
- [8. Comparativa y Siguientes Pasos](#8-comparativa-y-siguientes-pasos)

## 0. Contexto del Problema

El dataset `teratogenicity_data.txt` contiene medidas quimicas y una etiqueta binaria `Teratogenicity` que indica si una sustancia es teratogenica. El objetivo es construir modelos de clasificacion evaluados con AUC-ROC y accuracy, ademas de analizar la complejidad en terminos de features utilizados.

## 1. Preparacion del Entorno

### 1.1. Importacion de Librerias
Incluimos librerias para manipulacion de datos, modelado y utilidades.

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from IPython.display import display

from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

### 1.2. Configuracion General
Silenciamos warnings y fijamos semilla para reproducibilidad.

In [ ]:
warnings.filterwarnings("ignore")
SEED = 42
np.random.seed(SEED)

## 2. Comprension y Limpieza de Datos

### 2.1. Carga y Exploracion Inicial
Leemos el archivo, revisamos dimensiones y verificamos valores faltantes.

In [ ]:
DATA_PATH = os.path.join('.', 'data', 'teratogenicity_data.txt')
df = pd.read_csv(DATA_PATH).set_index('ID')

print(f'Forma del dataset: {df.shape}')
print('\nPrimeras filas:')
display(df.head())

print('\nResumen estadistico:')
display(df.describe())

missing = df.isnull().sum()
print('\nValores nulos por columna:')
if missing.sum() == 0:
    print('No se detectaron valores nulos')
else:
    print(missing[missing > 0])

### 2.2. Preparacion de Matrices X e y
Separamos features y target. La etiqueta usa valores {-1, 1}; la convertimos a {0, 1} para una clasificacion binaria estandar.

In [ ]:
y = df['Teratogenicity'].replace({-1: 0}).values
feature_cols = [col for col in df.columns if col != 'Teratogenicity']
X = df[feature_cols].values

print(f'Shape X: {X.shape}')
print(f'Shape y: {y.shape}')
print(f'Clases en y: {np.unique(y)}')

## 3. Preprocesamiento

### 3.1. Variance Threshold
Eliminamos columnas casi constantes porque aportan poca informacion discriminante.

In [ ]:
# Aplicamos un filtro básico para eliminar columnas casi constantes, lo que no aporta información al modelo
var_selector = VarianceThreshold(threshold=0.5)
var_selector.fit(X)

variance_mask = var_selector.get_support()
feature_names_reduced = np.array(feature_cols)[variance_mask]

print(f'Features originales: {X.shape[1]}')
print(f'Features tras filtrar por varianza: {feature_names_reduced.size}')

### 3.2. Estandarizacion
La estandarización alinea las escalas de los predictores. Ajustaremos el `StandardScaler` únicamente con los datos de entrenamiento para evitar fuga de información.

In [ ]:
scaler = StandardScaler()
print('Scaler instanciado; se ajustará tras crear el conjunto de entrenamiento.')

## 4. Split de Entrenamiento y Test
Dividimos el dataset al 50% manteniendo la proporción de clases (`stratify`). También aplicamos el filtro de varianza y el escalado usando únicamente información del entrenamiento.

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.5,
    stratify=y,
    random_state=SEED,
)

var_selector_train = VarianceThreshold(threshold=var_selector.threshold)
var_selector_train.fit(X_train_raw)

train_mask = var_selector_train.get_support()
selected_features = np.array(feature_cols)[train_mask]

X_train_filtered = var_selector_train.transform(X_train_raw)
X_test_filtered = var_selector_train.transform(X_test_raw)

X_train = scaler.fit_transform(X_train_filtered)
X_test = scaler.transform(X_test_filtered)

print(f"Tamaño X_train: {X_train.shape}")
print(f"Tamaño X_test: {X_test.shape}")
print(f"Features seleccionadas tras preprocesado: {selected_features.size}")

## 5. Modelo 1: Logistic Regression con Lasso
Entrenamos una regresión logística con regularización L1 (`penalty='l1'`) para inducir sparsity. Utilizamos `LogisticRegressionCV` para seleccionar automáticamente la intensidad de regularización (`C`) optimizando AUC-ROC mediante validación cruzada.

In [ ]:
lasso_clf = LogisticRegressionCV(
    Cs=10,
    cv=5,
    penalty='l1',
    solver='saga',
    scoring='roc_auc',
    max_iter=5000,
    n_jobs=-1,
    random_state=SEED,
).fit(X_train, y_train)

lasso_train_proba = lasso_clf.predict_proba(X_train)[:, 1]
lasso_test_proba = lasso_clf.predict_proba(X_test)[:, 1]

lasso_metrics = {
    'cv_mean_auc': lasso_clf.scores_[1].mean(axis=0)[np.where(lasso_clf.Cs_ == lasso_clf.C_[0])[0][0]],
    'train_auc': roc_auc_score(y_train, lasso_train_proba),
    'test_auc': roc_auc_score(y_test, lasso_test_proba),
    'train_accuracy': accuracy_score(y_train, lasso_clf.predict(X_train)),
    'test_accuracy': accuracy_score(y_test, lasso_clf.predict(X_test)),
    'non_zero_coef': np.count_nonzero(lasso_clf.coef_),
}

print('Resultados Lasso:')
for k, v in lasso_metrics.items():
    print(f'  {k}: {v:.4f}' if isinstance(v, float) else f'  {k}: {v}')

## 6. Modelo 2: Logistic Regression con Elastic Net
El modelo Elastic Net combina penalizaciones L1 y L2. Evaluamos una rejilla de `l1_ratio` para equilibrar sparsity y estabilidad, de nuevo optimizando AUC-ROC con validación cruzada.

In [ ]:
elastic_clf = LogisticRegressionCV(
    Cs=10,
    cv=5,
    penalty='elasticnet',
    solver='saga',
    l1_ratios=np.linspace(0.1, 0.9, 9),
    scoring='roc_auc',
    max_iter=5000,
    n_jobs=-1,
    random_state=SEED,
).fit(X_train, y_train)

elastic_train_proba = elastic_clf.predict_proba(X_train)[:, 1]
elastic_test_proba = elastic_clf.predict_proba(X_test)[:, 1]

elastic_scores = elastic_clf.scores_[1].mean(axis=0)
best_c_idx = np.where(elastic_clf.Cs_ == elastic_clf.C_[0])[0][0]
best_l1_idx = np.where(elastic_clf.l1_ratios_ == elastic_clf.l1_ratio_[0])[0][0]

elastic_metrics = {
    'cv_mean_auc': elastic_scores[best_c_idx, best_l1_idx],
    'train_auc': roc_auc_score(y_train, elastic_train_proba),
    'test_auc': roc_auc_score(y_test, elastic_test_proba),
    'train_accuracy': accuracy_score(y_train, elastic_clf.predict(X_train)),
    'test_accuracy': accuracy_score(y_test, elastic_clf.predict(X_test)),
    'non_zero_coef': np.count_nonzero(elastic_clf.coef_),
    'best_l1_ratio': elastic_clf.l1_ratio_[0],
}

print('Resultados Elastic Net:')
for k, v in elastic_metrics.items():
    print(f'  {k}: {v:.4f}' if isinstance(v, float) else f'  {k}: {v}')

## 7. Seleccion de Features con SelectKBest
Probamos distintos valores de `k` con una prueba univariada ANOVA (`f_classif`). Para cada `k`, ajustamos una regresión logística, medimos el AUC-ROC medio en validación cruzada y seleccionamos la mejor combinación.

In [ ]:
max_k = min(50, X_train.shape[1])
k_grid = range(1, max_k + 1)

cv_auc_per_k = []
base_logreg = LogisticRegression(max_iter=5000, solver='lbfgs')

for k in k_grid:
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_k = selector.fit_transform(X_train, y_train)
    scores = cross_val_score(
        base_logreg,
        X_train_k,
        y_train,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
    )
    cv_auc_per_k.append(scores.mean())

best_idx = int(np.argmax(cv_auc_per_k))
best_k = list(k_grid)[best_idx]

print(f'Mejor k por validación cruzada: {best_k}')

In [ ]:
cv_auc_df = pd.DataFrame({'k': list(k_grid), 'cv_auc': cv_auc_per_k})
display(cv_auc_df.head(10))
display(cv_auc_df.tail(10))

In [ ]:
best_selector = SelectKBest(score_func=f_classif, k=best_k)
X_train_best = best_selector.fit_transform(X_train, y_train)
X_test_best = best_selector.transform(X_test)

selected_feature_names = selected_features[best_selector.get_support()]

fs_clf = LogisticRegression(max_iter=5000, solver='lbfgs').fit(X_train_best, y_train)

fs_train_proba = fs_clf.predict_proba(X_train_best)[:, 1]
fs_test_proba = fs_clf.predict_proba(X_test_best)[:, 1]

feature_selection_metrics = {
    'cv_mean_auc': cv_auc_per_k[best_idx],
    'train_auc': roc_auc_score(y_train, fs_train_proba),
    'test_auc': roc_auc_score(y_test, fs_test_proba),
    'train_accuracy': accuracy_score(y_train, fs_clf.predict(X_train_best)),
    'test_accuracy': accuracy_score(y_test, fs_clf.predict(X_test_best)),
    'selected_features': selected_feature_names.size,
}

print('Resultados Logistic Regression + SelectKBest:')
for k, v in feature_selection_metrics.items():
    print(f'  {k}: {v:.4f}' if isinstance(v, float) else f'  {k}: {v}')

print('\nFeatures retenidas:')
print(list(selected_feature_names))

## 8. Comparativa y Siguientes Pasos
Resumimos los resultados clave para contrastar regularizaciones y selección de variables. Este cuadro facilita identificar el compromiso entre desempeño, estabilidad y complejidad.

In [ ]:
comparison_df = pd.DataFrame([
    {
        'modelo': 'Logistic L1',
        'cv_mean_auc': lasso_metrics['cv_mean_auc'],
        'test_auc': lasso_metrics['test_auc'],
        'test_accuracy': lasso_metrics['test_accuracy'],
        'features_activas': lasso_metrics['non_zero_coef'],
    },
    {
        'modelo': 'Logistic Elastic Net',
        'cv_mean_auc': elastic_metrics['cv_mean_auc'],
        'test_auc': elastic_metrics['test_auc'],
        'test_accuracy': elastic_metrics['test_accuracy'],
        'features_activas': elastic_metrics['non_zero_coef'],
    },
    {
        'modelo': f'Logistic + SelectKBest (k={best_k})',
        'cv_mean_auc': feature_selection_metrics['cv_mean_auc'],
        'test_auc': feature_selection_metrics['test_auc'],
        'test_accuracy': feature_selection_metrics['test_accuracy'],
        'features_activas': feature_selection_metrics['selected_features'],
    },
])

comparison_df

### Ideas para profundizar
- Explorar `Pipeline` o `ColumnTransformer` para encapsular preprocesos y reducir fugas.
- Revisar métricas adicionales (precisión, recall, matriz de confusión) según los requisitos del dominio.
- Probar otras técnicas de selección o regularización (ej. `RecursiveFeatureElimination`, `XGBoost`).